In [ ]:
import pandas as pd
import numpy as np

final_merged_df = pd.read_csv("VIP_30k_merged_data_59364_822.csv")

# 1. 데이터 정렬 (회원별, 시간순)
df = final_merged_df.sort_values(by=['발급회원번호', '기준년월']).copy()

# 2. 이용금액 합산 및 평균 계산
# 당월 총 이용금액 (신용 + 체크)
df['당월_총_이용금액'] = df['이용금액_신용_B0M'] + df['이용금액_체크_B0M']

# 직전 3개월 평균 이용금액 (신용 R3M + 체크 R3M) / 3
df['직전_3M_평균_이용금액'] = (df['이용금액_신용_R3M'] + df['이용금액_체크_R3M']) / 3

# 3. 이탈 타겟 정의 로직 (Method B)
def define_churn_strict(row):
    # 직전 3개월 평균이 없거나 0원인 경우 -> '판단 제외(휴면/신규)'로 분류
    if pd.isna(row['직전_3M_평균_이용금액']) or row['직전_3M_평균_이용금액'] <= 0:
        return np.nan 
    
    # 직전 평균 대비 10% 미만 사용 시 이탈(1), 아니면 유지(0)
    if row['당월_총_이용금액'] < (row['직전_3M_평균_이용금액'] * 0.1):
        return 1
    else:
        return 0

df['이탈_타겟'] = df.apply(define_churn_strict, axis=1)

# 4. 데이터 분리 (학습용 vs 휴면/신규 제외용)
# 타겟이 0 또는 1로 확실히 정해진 데이터 (학습용)
train_df = df[df['이탈_타겟'].notna()].copy()
train_df['이탈_타겟'] = train_df['이탈_타겟'].astype(int)

# 타겟이 NaN인 데이터 (별도 관리용: 장기 휴면 또는 신규 고객)
dormant_new_df = df[df['이탈_타겟'].isna()].copy()

# 5. 결과 확인
print("--- [데이터 분리 결과] ---")
print(f"1. 학습 가능 데이터(0, 1): {len(train_df)}건")
print(f"   - 유지(0): {len(train_df[train_df['이탈_타겟'] == 0])}건")
print(f"   - 이탈(1): {len(train_df[train_df['이탈_타겟'] == 1])}건")
print(f"\n2. 판단 제외 데이터(NaN): {len(dormant_new_df)}건 (장기 미사용/신규)")
print("--------------------------")

# 6. (선택) 각각 저장
# train_df.to_csv('3M_VIP_Target_data.csv', index=False)
# dormant_new_df.to_csv('dormant_customers.csv', index=False)

/var/folders/3p/8hc9c44s38j902ytq253kt6m0000gn/T/ipykernel_2146/161362198.py:4: DtypeWarning: Columns (552,637) have mixed types. Specify dtype option on import or set low_memory=False.
  final_merged_df = pd.read_csv("3M_VIP_Target_data.csv")


KeyboardInterrupt: 